In [5]:
# @markdown 💎 **Nyra AI Face Swap Studio V32**
import os
import time
import subprocess
import re
import cv2
import ipywidgets as widgets
from datetime import timedelta
from IPython.display import display, HTML, clear_output
from google.colab import drive
import shutil

# --- 1. GOOGLE DRIVE SETUP (DATA ONLY) ---
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# --- CONFIGURATION ---
# DATA PATHS -> GOOGLE DRIVE (Safe Storage)
DRIVE_ROOT = "/content/drive/MyDrive/Nyra_FaceSwap"
DEFAULT_SRC = os.path.join(DRIVE_ROOT, "Source_Images")
DEFAULT_TGT = os.path.join(DRIVE_ROOT, "Target_Videos")
DEFAULT_OUT = os.path.join(DRIVE_ROOT, "Outputs")

# ENGINE PATH -> LOCAL COLAB (Faster & Doesn't clutter Drive)
REPO_PATH = "/content/RealMe"
SCRIPT_PATH = os.path.join(REPO_PATH, "run.py")

for p in [DEFAULT_SRC, DEFAULT_TGT, DEFAULT_OUT]:
    os.makedirs(p, exist_ok=True)

# --- 2. CSS STYLING (YOUR PREFERRED STYLE) ---
style = """
<style>
    .hud-container {
        background-color: #050505; padding: 20px; border-radius: 25px;
        border: 1px solid #1f2833; max-width: 850px; margin: 0 auto;
        font-family: 'Segoe UI', monospace; color: #c5c6c7;
        box-shadow: 0 15px 50px rgba(0, 0, 0, 0.6);
        display: flex; flex-direction: column; gap: 15px;
    }
    .header-title {
        font-size: 22px; font-weight: 900; text-align: center; margin: 0; letter-spacing: 2px;
        background: -webkit-linear-gradient(0deg, #45a29e, #66fcf1);
        -webkit-background-clip: text; -webkit-text-fill-color: transparent;
    }
    .stats-strip {
        display: flex; justify-content: space-around; background: #0f0f0f;
        padding: 0 20px; border-radius: 50px; border: 1px solid #222;
        align-items: center; width: 100%; box-sizing: border-box; height: 50px;
    }
    .stat-label { font-size: 10px; color: #666; font-weight: 800; letter-spacing: 1px; margin-right: 8px; }
    .stat-value { font-size: 14px; color: #fff; font-weight: bold; }
    .divider { width: 1px; height: 20px; background: #333; }

    .progress-wrapper {
        background: #000; height: 10px; border-radius: 20px;
        border: 1px solid #333; position: relative; overflow: hidden; margin-bottom: 4px;
    }
    .progress-fill {
        height: 100%; width: 0%; border-radius: 20px;
        background: linear-gradient(90deg, #45a29e, #66fcf1);
        transition: width 0.3s ease-out;
    }
    .progress-meta { display: flex; justify-content: space-between; font-size: 10px; color: #888; font-weight: bold; padding: 0 5px; margin-bottom: 2px; }

    .styled-widget select, .styled-widget input {
        background-color: #0f0f0f !important; color: #ffffff !important;
        border: 1px solid #333 !important; border-radius: 20px !important;
        width: 100%; height: 30px !important; padding: 0 15px !important;
        font-family: monospace; font-size: 12px !important; transition: border 0.2s;
    }
    .styled-widget select:focus { border-color: #66fcf1 !important; outline: none; }
    .input-label-text {
        color: #666; font-size: 10px; font-weight: 800; letter-spacing: 1px;
        margin-bottom: 5px; margin-left: 15px;
    }

    .smart-btn {
        background: linear-gradient(135deg, #1f4037 0%, #66fcf1 100%);
        border: none; color: #000 !important; height: 50px; width: 100%;
        font-weight: 900; font-size: 14px; letter-spacing: 2px;
        border-radius: 25px; text-transform: uppercase; cursor: pointer;
        box-shadow: 0 4px 15px rgba(102, 252, 241, 0.15); transition: all 0.2s;
    }
    .smart-btn:hover { transform: translateY(-2px); box-shadow: 0 6px 20px rgba(102, 252, 241, 0.25); }
    .smart-btn.setup-mode { background: #1a1a1a; border: 2px solid #333; color: #fff !important; box-shadow: none; }

    .clean-btn {
        background: #2a0e0e; color: #ff6666; border: 1px solid #442222;
        border-radius: 20px; font-size: 10px; font-weight: bold; cursor: pointer; padding: 0 15px; height: 30px;
    }

    /* COLLAPSIBLE BUTTONS ARE BACK */
    .collapse-btn {
        background-color: #0f0f0f !important; color: #666 !important;
        border-radius: 20px !important; border: 1px solid #222 !important;
        font-weight: bold !important; font-size: 11px !important; height: 35px !important;
        text-align: center; margin-bottom: 0; width: 100%; transition: all 0.2s;
    }
    .collapse-btn:hover { color: #66fcf1 !important; border-color: #66fcf1 !important; }
    .collapse-btn.active { background-color: #1a1a1a !important; color: #66fcf1 !important; border-color: #66fcf1 !important; }

    .log-box {
        background: #080808; color: #0f0; font-family: 'Consolas', monospace;
        padding: 10px 15px; width: 100%; height: 120px; overflow-y: scroll;
        border: 1px solid #222; font-size: 10px; white-space: pre-wrap; border-radius: 15px; margin-top: 5px;
    }
    .dl-row { display: flex; justify-content: space-between; align-items: center; background: #0f0f0f; padding: 6px 15px; border-radius: 20px; margin-bottom: 5px; border: 1px solid #222; }
    .dl-filename { font-size: 11px; color: #66fcf1; font-family: monospace; font-weight: bold; }
    .dl-btn { background: #222; color: #fff; border: 1px solid #444; padding: 4px 0; width: 60px; border-radius: 20px; font-size: 9px; cursor: pointer; font-weight: bold; text-align:center; }
</style>
"""
display(HTML(style))

# --- 3. UTILITIES ---
def get_files(folder, exts):
    if not os.path.exists(folder): return []
    return sorted([f for f in os.listdir(folder) if f.lower().endswith(exts)])

def check_installation():
    # Check LOCAL engine
    return os.path.exists(SCRIPT_PATH) and os.path.exists(os.path.join(REPO_PATH, "checkpoints/inswapper_128.onnx"))

# --- 4. WIDGET SETUP ---
header = widgets.HTML('<div class="header-title">Nyra AI Face Swap Studio</div>')

# Stats
val_batch = widgets.HTML('<div class="stat-value" style="color:#e056fd">0 / 0</div>')
val_frames = widgets.HTML('<div class="stat-value" style="color:#66fcf1">WAITING</div>')
val_eta = widgets.HTML('<div class="stat-value" style="color:#38ef7d">--:--</div>')

stats_strip = widgets.HBox([
    widgets.HBox([widgets.HTML('<div class="stat-label">BATCH:</div>'), val_batch], layout=widgets.Layout(align_items='center', justify_content='center', width='30%')),
    widgets.HTML('<div class="divider"></div>'),
    widgets.HBox([widgets.HTML('<div class="stat-label">STATUS:</div>'), val_frames], layout=widgets.Layout(align_items='center', justify_content='center', width='30%')),
    widgets.HTML('<div class="divider"></div>'),
    widgets.HBox([widgets.HTML('<div class="stat-label">ETA:</div>'), val_eta], layout=widgets.Layout(align_items='center', justify_content='center', width='30%')),
])
stats_strip.add_class('stats-strip')

# Progress
bar_batch = widgets.HTML('<div class="progress-wrapper"><div class="progress-fill" style="width: 0%"></div></div>')
bar_file = widgets.HTML('<div class="progress-wrapper"><div class="progress-fill" style="width: 0%"></div></div>')
meta_batch = widgets.HTML('<div class="progress-meta"><span>BATCH TOTAL</span><span>0%</span></div>')
meta_file = widgets.HTML('<div class="progress-meta"><span>CURRENT FILE</span><span>IDLE</span></div>')

progress_row = widgets.HBox([
    widgets.VBox([meta_batch, bar_batch], layout=widgets.Layout(width='49%')),
    widgets.VBox([meta_file, bar_file], layout=widgets.Layout(width='49%'))
], layout=widgets.Layout(justify_content='space-between', width='100%'))

# Inputs
drop_src = widgets.Dropdown(options=get_files(DEFAULT_SRC, ('.jpg','.png')), description='')
drop_src.add_class('styled-widget')
drop_tgt = widgets.Dropdown(options=get_files(DEFAULT_TGT, ('.mp4','.mkv','.avi','.mov', '.jpg', '.png')) or ["NO FILES"], description='', disabled=False)
drop_tgt.add_class('styled-widget')

input_row = widgets.HBox([
    widgets.VBox([widgets.HTML('<div class="input-label-text">👤 FACE SOURCE (Drive)</div>'), drop_src], layout=widgets.Layout(width='49%')),
    widgets.VBox([widgets.HTML('<div class="input-label-text">🎬 TARGET FILE (Drive)</div>'), drop_tgt], layout=widgets.Layout(width='49%'))
], layout=widgets.Layout(justify_content='space-between', width='100%'))

# Options
chk_enhance = widgets.Checkbox(value=True, description='ENHANCE')
chk_delete = widgets.Checkbox(value=False, description='DELETE TARGET')
btn_clean = widgets.Button(description="CLEAN")
btn_clean.add_class('clean-btn')

opts_box = widgets.HBox([chk_enhance, chk_delete, btn_clean], layout=widgets.Layout(justify_content='space-between', width='100%', align_items='center'))

# Main Button
btn_main = widgets.Button(description="LOADING...")
btn_main.add_class('smart-btn')

# Collapsible Notes
def create_collapsible(title, content_widget):
    toggle_btn = widgets.Button(description=f"▶ {title}", layout=widgets.Layout(width='100%'))
    toggle_btn.add_class('collapse-btn')
    content_widget.layout.display = 'none'
    def on_toggle(b):
        if content_widget.layout.display == 'none':
            content_widget.layout.display = 'block'
            b.description = f"▼ {title}"
            b.add_class('active')
        else:
            content_widget.layout.display = 'none'
            b.description = f"▶ {title}"
            b.remove_class('active')
    toggle_btn.on_click(on_toggle)
    return widgets.VBox([toggle_btn, content_widget])

dl_output_box = widgets.VBox([])
log_out = widgets.Output()
log_out.add_class('log-box')

note_dl = create_collapsible("DOWNLOADS", dl_output_box)
note_logs = create_collapsible("SYSTEM LOGS", log_out)

notes_row = widgets.HBox([note_dl, note_logs], layout=widgets.Layout(width='100%', justify_content='space-between'))
note_dl.layout.width = '49%'
note_logs.layout.width = '49%'

# --- 5. LOGIC ---
SYSTEM_INSTALLED = check_installation()

def update_ui_state():
    global SYSTEM_INSTALLED
    if SYSTEM_INSTALLED:
        btn_main.description = "🚀 INITIATE FUSION"
        btn_main.remove_class('setup-mode')
    else:
        btn_main.description = "🛠️ SETUP ENGINE (ONE TIME)"
        btn_main.add_class('setup-mode')

update_ui_state()

def update_bar(html_widget, meta_widget, percent, label):
    color = "linear-gradient(90deg, #38ef7d, #11998e)" if "FRAME" in label or "SWAP" in label else "linear-gradient(90deg, #45a29e, #66fcf1)"
    html_widget.value = f'<div class="progress-wrapper"><div class="progress-fill" style="width: {percent}%; background: {color}"></div></div>'
    meta_widget.value = f'<div class="progress-meta"><span>{label}</span><span>{int(percent)}%</span></div>'

def add_download_btn(filepath):
    filename = os.path.basename(filepath)
    lbl = widgets.HTML(f'<div class="dl-filename">📄 {filename}</div>')
    btn = widgets.Button(description="SAVE", layout=widgets.Layout(width='40px'))
    btn.add_class('dl-btn')
    def on_dl_click(b):
        files.download(filepath)
        b.description = "..."
        b.disabled = True
    btn.on_click(on_dl_click)
    row = widgets.HBox([lbl, btn])
    row.add_class('dl-row')
    dl_output_box.children = [row] + list(dl_output_box.children)
    dl_output_box.layout.display = 'block' # Auto show if hidden

def run_installer():
    global SYSTEM_INSTALLED
    btn_main.disabled = True
    btn_main.description = "INSTALLING ENGINE..."
    log_out.layout.display = 'block'
    log_out.clear_output()

    def log(msg):
        with log_out: print(f"[SETUP] {msg}")

    try:
        log("Updating Env...")
        update_bar(bar_file, meta_file, 10, "ENV UPDATE")
        os.system("apt-get update -qq && apt-get install -y ffmpeg -qq")

        log("Cloning Local Engine...")
        update_bar(bar_file, meta_file, 30, "CLONING")
        if os.path.exists(REPO_PATH): shutil.rmtree(REPO_PATH)
        os.system(f"git clone https://github.com/RixAI/RealMe.git \"{REPO_PATH}\"")

        log("Deps...")
        update_bar(bar_file, meta_file, 50, "DEPS")
        os.system('pip install "numpy<2.0.0" tensorflow onnxruntime-gpu insightface==0.7.3 psutil gfpgan basicsr rich opencv-python imageio-ffmpeg')

        log("Patching...")
        update_bar(bar_file, meta_file, 70, "PATCHING")
        os.system("sed -i 's/from torchvision.transforms.functional_tensor import rgb_to_grayscale/from torchvision.transforms.functional import rgb_to_grayscale/g' /usr/local/lib/python*/dist-packages/basicsr/data/degradations.py")

        log("Models...")
        update_bar(bar_file, meta_file, 90, "MODELS")
        ckpt_path = os.path.join(REPO_PATH, "checkpoints")
        os.makedirs(ckpt_path, exist_ok=True)
        os.system(f"wget -nc -q https://huggingface.co/ezioruan/inswapper_128.onnx/resolve/main/inswapper_128.onnx -O \"{os.path.join(ckpt_path, 'inswapper_128.onnx')}\"")
        os.system(f"wget -nc -q https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -O \"{os.path.join(ckpt_path, 'GFPGANv1.3.pth')}\"")

        update_bar(bar_file, meta_file, 100, "READY")
        log("DONE!")
        SYSTEM_INSTALLED = True
        update_ui_state()
        btn_main.disabled = False
    except:
        log("ERROR")
        btn_main.disabled = False

def run_batch_process():
    targets = get_files(DEFAULT_TGT, ('.mp4','.mkv','.avi','.mov', '.jpg', '.png'))
    if not drop_src.value or not targets:
        with log_out: print("❌ NO FILES FOUND IN DRIVE FOLDERS")
        return

    btn_main.disabled = True
    btn_main.description = "PROCESSING..."
    log_out.clear_output()
    dl_output_box.children = []

    src_path = os.path.join(DEFAULT_SRC, drop_src.value)
    total_files = len(targets)

    for idx, tgt_file in enumerate(targets):
        # FIX: Batch Logic was visual only, now tracks idx
        current_batch_num = idx + 1

        # UI UPDATE - Update Batch & File Name BEFORE processing
        update_bar(bar_batch, meta_batch, (idx / total_files) * 100, f"BATCH ({current_batch_num}/{total_files})")
        val_batch.value = f'<div class="stat-value" style="color:#e056fd">{current_batch_num} / {total_files}</div>'
        val_frames.value = f'<div class="stat-value" style="color:#66fcf1">INIT</div>'
        meta_file.value = f'<div class="progress-meta"><span>{tgt_file}</span><span>INIT</span></div>'

        tgt_path = os.path.join(DEFAULT_TGT, tgt_file)
        name, ext = os.path.splitext(tgt_file)
        is_video = ext.lower() in ['.mp4', '.mkv', '.avi', '.mov']
        out_path = os.path.join(DEFAULT_OUT, f"{name}_Fusion{ext}")

        cmd = ["python", SCRIPT_PATH, "-s", src_path, "-t", tgt_path, "-o", out_path, "--execution-provider", "cuda"]
        if is_video: cmd.extend(["--temp-frame-quality", "1", "--output-video-quality", "1"])

        processors = ["face_swapper"]
        if chk_enhance.value: processors.append("face_enhancer")
        cmd.extend(["--frame-processor"] + processors)

        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, universal_newlines=True)

        step_start = time.time()
        curr_frames = 0
        tot_frames = 1

        while True:
            line = process.stdout.readline()
            if not line and process.poll() is not None: break
            if line:
                raw = line.strip()
                with log_out: print(raw)
                match = re.search(r"(\d+)/(\d+)", raw)
                if match and "Processing" in raw:
                    curr_frames = int(match.group(1))
                    tot_frames = int(match.group(2))

                    elapsed = time.time() - step_start
                    if elapsed > 0 and curr_frames > 0: fps = curr_frames / elapsed
                    else: fps = 0.1

                    rem = max(0, tot_frames - curr_frames)
                    eta = str(timedelta(seconds=int(rem / fps)))

                    val_frames.value = f'<div class="stat-value" style="color:#66fcf1">{curr_frames}/{tot_frames}</div>'
                    val_eta.value = f'<div class="stat-value" style="color:#38ef7d">{eta}</div>'

                    # Update file bar with name
                    pct = (curr_frames/tot_frames)*100
                    update_bar(bar_file, meta_file, pct, f"{tgt_file} ({int(pct)}%)")

        update_bar(bar_file, meta_file, 100, f"{tgt_file} (DONE)")
        if os.path.exists(out_path):
            add_download_btn(out_path)
            if chk_delete.value:
                try: os.remove(tgt_path)
                except: pass

    update_bar(bar_batch, meta_batch, 100, "ALL DONE")
    val_batch.value = f'<div class="stat-value" style="color:#e056fd">{total_files}/{total_files}</div>'
    val_frames.value = '<div class="stat-value" style="color:#38ef7d">DONE</div>'
    val_eta.value = '<div class="stat-value">00:00</div>'
    btn_main.disabled = False
    btn_main.description = "🚀 INITIATE FUSION"

    # Refresh lists
    drop_tgt.options = get_files(DEFAULT_TGT, ('.mp4','.mkv','.avi','.mov', '.jpg', '.png')) or ["NO FILES"]

def on_main_click(b):
    if not SYSTEM_INSTALLED: run_installer()
    else: run_batch_process()

def on_clean_click(b):
    for folder in [DEFAULT_TGT, DEFAULT_OUT]:
        if os.path.exists(folder): shutil.rmtree(folder); os.makedirs(folder)
    drop_tgt.options = ["CLEANED"]
    dl_output_box.children = []
    with log_out: print("🧹 CLEANED")

btn_main.on_click(on_main_click)
btn_clean.on_click(on_clean_click)

# --- 6. DISPLAY ---
drop_src.options = get_files(DEFAULT_SRC, ('.jpg','.png'))
drop_tgt.options = get_files(DEFAULT_TGT, ('.mp4','.mkv','.avi','.mov', '.jpg', '.png')) or ["NO FILES"]

ui = widgets.VBox([
    header,
    stats_strip,
    progress_row,
    input_row,
    opts_box,
    btn_main,
    notes_row
])
ui.add_class('hud-container')

display(ui)

#### **Devloper**

In [ ]:
# @title 🛠️ **1. Install RealMe Engine**
import ipywidgets as widgets
from IPython.display import display, HTML
import subprocess
import os
import sys
import shutil

# --- CSS STYLE ---
style = """
<style>
    .install-card {
        background-color: #0d1117; border-radius: 20px; padding: 20px;
        border: 1px solid #30363d; font-family: 'Roboto', sans-serif;
        color: #c9d1d9; box-shadow: 0 10px 30px rgba(0,0,0,0.5);
    }
    .title { font-size: 20px; font-weight: 800; color: #fff; margin-bottom: 15px; letter-spacing: 1px; }
    .status-box { background: #161b22; border-radius: 10px; padding: 10px; font-family: monospace; font-size: 12px; color: #8b949e; border: 1px solid #30363d; }
    .success { color: #2ea043; font-weight: bold; }
    .error { color: #da3633; font-weight: bold; }
    .bar-bg { background: #21262d; height: 6px; border-radius: 10px; overflow: hidden; margin-top: 10px; }
    .bar-fill { height: 100%; width: 0%; background: linear-gradient(90deg, #a371f7, #6e40c9); transition: width 0.4s; }
</style>
"""
display(HTML(style))

# --- UI SETUP ---
title = widgets.HTML('<div class="title">🚀 REALME ENGINE SETUP</div>')
log_box = widgets.HTML('<div class="status-box">Waiting to start...</div>')
progress_html = widgets.HTML('<div class="bar-bg"><div id="p-bar" class="bar-fill" style="width: 0%;"></div></div>')
ui = widgets.VBox([title, log_box, progress_html])
ui.add_class('install-card')
display(ui)

def update_log(msg, color="#8b949e"):
    log_box.value = f'<div class="status-box" style="color:{color}">{msg}</div>'

def update_progress(percent):
    progress_html.value = f'<div class="bar-bg"><div class="bar-fill" style="width: {percent}%;"></div></div>'

def run_step(cmd, desc, step_perc):
    update_log(f"⚙️ {desc}...", "#58a6ff")
    if os.system(cmd) != 0:
        update_log(f"❌ Error in {desc}", "#da3633")
        return False
    update_progress(step_perc)
    return True

# --- EXECUTION ---
try:
    # 1. System
    if not run_step("apt-get update -qq && apt-get install -y ffmpeg -qq", "Installing FFmpeg", 20): raise Exception("FFmpeg Failed")

    # 2. Clone
    if os.path.exists("RealMe"): shutil.rmtree("RealMe")
    if not run_step("git clone https://github.com/RixAI/RealMe.git", "Cloning Repository", 40): raise Exception("Git Clone Failed")

    # 3. Libs (Colab Safe)
    update_log("📦 Installing Libraries (This takes time)...", "#e3b341")
    os.system("pip uninstall -y numpy")
    deps = 'pip install "numpy<2.0.0" tensorflow onnxruntime-gpu insightface==0.7.3 psutil gfpgan basicsr rich opencv-python imageio-ffmpeg'
    if os.system(deps) != 0: raise Exception("Pip Install Failed")
    update_progress(70)

    # 4. Patch
    patch_cmd = "sed -i 's/from torchvision.transforms.functional_tensor import rgb_to_grayscale/from torchvision.transforms.functional import rgb_to_grayscale/g' /usr/local/lib/python*/dist-packages/basicsr/data/degradations.py"
    run_step(patch_cmd, "Applying Code Patches", 80)

    # 5. Models
    os.chdir("RealMe")
    os.makedirs("checkpoints", exist_ok=True)
    os.system("wget -nc -q https://huggingface.co/ezioruan/inswapper_128.onnx/resolve/main/inswapper_128.onnx -O checkpoints/inswapper_128.onnx")
    os.system("wget -nc -q https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -O checkpoints/GFPGANv1.3.pth")

    # Done
    import numpy
    update_progress(100)
    update_log(f"✅ SETUP COMPLETE | Numpy: {numpy.__version__}", "#2ea043")

except Exception as e:
    update_log(f"❌ CRITICAL FAILURE: {str(e)}", "#da3633")

In [ ]:
# @title 💎 **REALME STUDIO V12 (Stable & Zero-Error)**
import os
import time
import subprocess
import re
import ipywidgets as widgets
from datetime import timedelta
from IPython.display import display, HTML, clear_output
from google.colab import files

# --- 1. CONFIGURATION ---
DEFAULT_SRC = "/content/realme_workspace/source"
DEFAULT_TGT = "/content/realme_workspace/targets"
DEFAULT_OUT = "/content/realme_workspace/outputs"
for p in [DEFAULT_SRC, DEFAULT_TGT, DEFAULT_OUT]: os.makedirs(p, exist_ok=True)

# --- 2. CSS STYLING ---
style = """
<style>
    .hud-container {
        background-color: #050505; padding: 20px; border-radius: 16px;
        border: 1px solid #1f2833; max-width: 850px; margin: 0 auto;
        font-family: 'Segoe UI', monospace; color: #c5c6c7;
        box-shadow: 0 0 40px rgba(0, 255, 255, 0.05);
        display: flex; flex-direction: column; gap: 12px;
    }
    .stats-grid { display: grid; grid-template-columns: 1fr 1fr 1fr; gap: 10px; margin-bottom: 5px; }
    .stat-box {
        background: #111; padding: 10px; border-radius: 12px;
        border-left: 3px solid #45a29e; position: relative;
    }
    .stat-label { font-size: 9px; color: #666; letter-spacing: 1px; font-weight: bold; text-transform: uppercase; }
    .stat-value { font-size: 15px; color: #fff; font-weight: bold; margin-top: 3px; white-space: nowrap; }

    .progress-wrapper {
        background: #000; height: 14px; border-radius: 50px;
        border: 1px solid #333; position: relative; overflow: hidden; margin-bottom: 2px;
    }
    .progress-fill {
        height: 100%; width: 0%; border-radius: 50px;
        background: linear-gradient(90deg, #45a29e, #66fcf1);
        transition: width 0.3s ease-out;
    }
    .progress-meta { display: flex; justify-content: space-between; font-size: 9px; color: #888; font-weight: bold; padding: 0 5px; margin-bottom: 3px; }

    .styled-widget select, .styled-widget input {
        background: #111 !important; color: #fff !important;
        border: 1px solid #333 !important; padding: 8px !important;
        border-radius: 12px !important; width: 100%; font-family: monospace; font-size: 12px !important;
    }
    .styled-widget select:focus { border-color: #66fcf1 !important; outline: none; }

    .action-btn {
        background: linear-gradient(135deg, #1f4037 0%, #66fcf1 100%);
        border: none; color: #000 !important; height: 45px; width: 100%;
        font-weight: 900; font-size: 14px; letter-spacing: 2px;
        border-radius: 50px; text-transform: uppercase; cursor: pointer;
        box-shadow: 0 0 15px rgba(102, 252, 241, 0.2);
    }
    .action-btn:disabled { background: #333; color: #777 !important; cursor: not-allowed; box-shadow: none; }

    .log-box {
        background: #080808; color: #0f0; font-family: 'Consolas', monospace;
        padding: 10px; width: 100%; height: 150px; overflow-y: scroll;
        border: none; font-size: 10px; white-space: pre-wrap; border-radius: 0 0 12px 12px;
    }
    .dl-row { display: flex; justify-content: space-between; align-items: center; background: #111; padding: 6px 12px; border-radius: 10px; margin-bottom: 4px; border: 1px solid #222; }
    .dl-filename { font-size: 11px; color: #66fcf1; font-family: monospace; }
    .dl-btn { background: #222; color: #fff; border: 1px solid #444; padding: 4px 10px; border-radius: 15px; font-size: 9px; cursor: pointer; }
    .dl-btn:hover { background: #333; border-color: #66fcf1; }
</style>
"""
display(HTML(style))

# --- 3. UTILITIES ---
def get_files(folder, exts):
    if not os.path.exists(folder): return []
    return sorted([f for f in os.listdir(folder) if f.lower().endswith(exts)])

# --- 4. WIDGET SETUP ---
val_batch = widgets.HTML('<div class="stat-value">0 / 0</div>')
val_frames = widgets.HTML('<div class="stat-value">WAITING</div>')
val_eta = widgets.HTML('<div class="stat-value">--:--</div>')

stat_boxes = widgets.HBox([
    widgets.HTML(f'<div class="stat-box"><div class="stat-label" style="color:#e056fd">BATCH QUEUE</div>'),
    widgets.HTML(f'<div class="stat-box"><div class="stat-label" style="color:#66fcf1">FRAMES PROCESSED</div>'),
    widgets.HTML(f'<div class="stat-box"><div class="stat-label" style="color:#38ef7d">ETA</div>')
])

grid_html = widgets.HBox([
    widgets.VBox([stat_boxes.children[0], val_batch]),
    widgets.VBox([stat_boxes.children[1], val_frames]),
    widgets.VBox([stat_boxes.children[2], val_eta])
])
grid_html.add_class('stats-grid')

bar_batch = widgets.HTML('<div class="progress-wrapper"><div class="progress-fill" style="width: 0%"></div></div>')
bar_file = widgets.HTML('<div class="progress-wrapper"><div class="progress-fill" style="width: 0%"></div></div>')
meta_batch = widgets.HTML('<div class="progress-meta"><span>BATCH PROGRESS</span><span>0%</span></div>')
meta_file = widgets.HTML('<div class="progress-meta"><span>FILE STATUS</span><span>IDLE</span></div>')

progress_row = widgets.HBox([
    widgets.VBox([meta_batch, bar_batch], layout=widgets.Layout(width='49%')),
    widgets.VBox([meta_file, bar_file], layout=widgets.Layout(width='49%'))
], layout=widgets.Layout(justify_content='space-between', width='100%'))

drop_src = widgets.Dropdown(options=get_files(DEFAULT_SRC, ('.jpg','.png')), description='')
drop_src.add_class('styled-widget')
drop_tgt = widgets.Dropdown(options=get_files(DEFAULT_TGT, ('.mp4','.mkv','.avi','.mov', '.jpg', '.png')) or ["NO FILES"], description='', disabled=False)
drop_tgt.add_class('styled-widget')

input_row = widgets.HBox([
    widgets.VBox([widgets.Label("👤 FACE SOURCE:", style={'text_color':'#666', 'font_weight':'bold', 'font_size':'10px'}), drop_src], layout=widgets.Layout(width='49%')),
    widgets.VBox([widgets.Label("📂 TARGET PREVIEW:", style={'text_color':'#666', 'font_weight':'bold', 'font_size':'10px'}), drop_tgt], layout=widgets.Layout(width='49%'))
], layout=widgets.Layout(justify_content='space-between', width='100%'))

chk_enhance = widgets.Checkbox(value=True, description='ENHANCE (HD)')
chk_delete = widgets.Checkbox(value=False, description='DELETE SOURCE AFTER SUCCESS')
opts_box = widgets.HBox([chk_enhance, chk_delete])

btn_run = widgets.Button(description="INITIATE FUSION")
btn_run.add_class('action-btn')

dl_output_box = widgets.VBox([])
acc_dl = widgets.Accordion(children=[dl_output_box])
acc_dl.set_title(0, '📥 DOWNLOAD CENTER')

log_out = widgets.Output()
log_out.add_class('log-box')
acc_logs = widgets.Accordion(children=[log_out])
acc_logs.set_title(0, '📝 SYSTEM LOGS')

# --- 5. LOGIC & EXECUTION ---
def update_bar(html_widget, meta_widget, percent, label):
    color = "linear-gradient(90deg, #38ef7d, #11998e)" if "FRAME" in label or "SWAP" in label else "linear-gradient(90deg, #45a29e, #66fcf1)"
    html_widget.value = f'<div class="progress-wrapper"><div class="progress-fill" style="width: {percent}%; background: {color}"></div></div>'
    meta_widget.value = f'<div class="progress-meta"><span>{label}</span><span>{int(percent)}%</span></div>'

def add_download_btn(filepath):
    filename = os.path.basename(filepath)
    lbl = widgets.HTML(f'<div class="dl-filename">📄 {filename}</div>')
    btn = widgets.Button(description="GET", layout=widgets.Layout(width='60px'))
    btn.add_class('dl-btn')
    def on_dl_click(b):
        files.download(filepath)
        b.description = "..."
        b.disabled = True
    btn.on_click(on_dl_click)
    row = widgets.HBox([lbl, btn])
    row.add_class('dl-row')
    dl_output_box.children = [row] + list(dl_output_box.children)

def run_batch(b):
    targets = get_files(DEFAULT_TGT, ('.mp4','.mkv','.avi','.mov', '.jpg', '.png'))
    if not drop_src.value: return
    if not targets:
        with log_out: print("❌ NO FILES FOUND!")
        return

    btn_run.disabled = True
    btn_run.description = "SYSTEM RUNNING..."
    log_out.clear_output()
    dl_output_box.children = []

    src_path = os.path.join(DEFAULT_SRC, drop_src.value)
    total_files = len(targets)

    for idx, tgt_file in enumerate(targets):
        update_bar(bar_batch, meta_batch, (idx / total_files) * 100, f"BATCH ({idx}/{total_files})")
        val_batch.value = f'<div class="stat-value">{idx} / {total_files}</div>'
        val_frames.value = f'<div class="stat-value">INITIALIZING...</div>'

        tgt_path = os.path.join(DEFAULT_TGT, tgt_file)
        name, ext = os.path.splitext(tgt_file)
        is_video = ext.lower() in ['.mp4', '.mkv', '.avi', '.mov']
        out_path = os.path.join(DEFAULT_OUT, f"{name}_Fusion{ext}")

        # Command
        cmd = ["python", "run.py", "-s", src_path, "-t", tgt_path, "-o", out_path, "--execution-provider", "cuda"]
        if is_video: cmd.extend(["--temp-frame-quality", "1", "--output-video-quality", "1"])

        processors = ["face_swapper"]
        if chk_enhance.value: processors.append("face_enhancer")
        cmd.extend(["--frame-processor"] + processors)

        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, universal_newlines=True)

        active_phase = "SWAPPER"
        step_start_time = time.time()
        current_frames = 0
        total_frames = 1

        while True:
            line = process.stdout.readline()
            if not line and process.poll() is not None: break
            if line:
                raw = line.strip()
                with log_out: print(raw)

                if "FACE-SWAPPER" in raw:
                    active_phase = "SWAPPER"
                    step_start_time = time.time()
                elif "FACE-ENHANCER" in raw:
                    active_phase = "ENHANCER"
                    step_start_time = time.time()

                match = re.search(r"(\d+)/(\d+)", raw)

                if match and "Processing" in raw:
                    current_frames = int(match.group(1))
                    total_frames = int(match.group(2))

                    elapsed = time.time() - step_start_time

                    # --- SAFETY FIX HERE ---
                    if elapsed > 0 and current_frames > 0:
                         fps = current_frames / elapsed
                    else:
                         fps = 0.1 # Prevents ZeroDivision

                    rem_frames = max(0, total_frames - current_frames)
                    eta_seconds = rem_frames / fps
                    eta_str = str(timedelta(seconds=int(eta_seconds)))

                    val_frames.value = f'<div class="stat-value" style="color:#66fcf1">{current_frames} / {total_frames}</div>'
                    val_eta.value = f'<div class="stat-value">{eta_str}</div>'

                    file_pct = (current_frames / total_frames) * 100

                    if chk_enhance.value:
                        if active_phase == "SWAPPER":
                            display_pct = file_pct / 2
                            status_txt = f"SWAPPING ({int(file_pct)}%)"
                        else:
                            display_pct = 50 + (file_pct / 2)
                            status_txt = f"ENHANCING ({int(file_pct)}%)"
                    else:
                        display_pct = file_pct
                        status_txt = f"PROCESSING ({int(file_pct)}%)"

                    update_bar(bar_file, meta_file, display_pct, status_txt)

                elif "Creating video" in raw:
                    val_frames.value = f'<div class="stat-value" style="color:#38ef7d">FINALIZING...</div>'
                    update_bar(bar_file, meta_file, 99, "COMPILING")

        update_bar(bar_file, meta_file, 100, "DONE")
        if os.path.exists(out_path):
            add_download_btn(out_path)
            if chk_delete.value:
                try: os.remove(tgt_path)
                except: pass

    update_bar(bar_batch, meta_batch, 100, "COMPLETE")
    val_batch.value = f'<div class="stat-value">{total_files} / {total_files}</div>'
    val_frames.value = '<div class="stat-value" style="color:#38ef7d">ALL DONE</div>'
    val_eta.value = '<div class="stat-value">00:00</div>'
    btn_run.disabled = False
    btn_run.description = "INITIATE FUSION"

    leftover = get_files(DEFAULT_TGT, ('.mp4','.mkv','.avi','.mov', '.jpg', '.png'))
    drop_tgt.options = leftover if leftover else ["FOLDER EMPTY"]

btn_run.on_click(run_batch)

# --- 6. DISPLAY ---
drop_src.options = get_files(DEFAULT_SRC, ('.jpg','.png'))
drop_tgt.options = get_files(DEFAULT_TGT, ('.mp4','.mkv','.avi','.mov', '.jpg', '.png')) or ["NO FILES"]

ui = widgets.VBox([
    grid_html,
    progress_row,
    input_row,
    opts_box,
    btn_run,
    acc_dl,
    acc_logs
])
ui.add_class('hud-container')

display(ui)

In [ ]:
# @title 🧹 **Cleanup**

import ipywidgets as widgets
from IPython.display import display
import shutil
import os

btn_clean = widgets.Button(description="🗑️ DELETE VIDEOS", button_style='danger')
out = widgets.Output()

display(btn_clean, out)

def clean_folders(b):
    targets = ["/content/my_videos", "/content/output_videos"]
    with out:
        out.clear_output()
        print("Cleaning...")
        for t in targets:
            if os.path.exists(t):
                try:
                    shutil.rmtree(t)
                    os.makedirs(t)
                    print(f"✅ Reset: {t}")
                except Exception as e:
                    print(f"❌ Error: {e}")
            else:
                os.makedirs(t)
                print(f"✨ Created: {t}")
        print("Done!")

btn_clean.on_click(clean_folders)

In [ ]:
!python run.py -s "/content/realme_workspace/source/unnamed.png" -t "/content/realme_workspace/targets/One.mp4" -o "/content/realme_workspace/outputs/debug_output.mp4" --execution-provider cuda --frame-processor face_swapper face_enhancer --temp-frame-quality 1 --output-video-quality 1